In [34]:
import numpy as np
import pandas as pd

import model

In [37]:
length = 120.
num_rep = 20
reorder_point = 30
order_size = 60

df = pd.DataFrame(model.run_experiments([reorder_point], [order_size], num_rep))
df

,reorder_point,order_size,total_cost,ordering_cost,holding_cost,shortage_cost
0,30,60,120.2,81.0,35.0,4.2
1,30,60,124.1,83.2,36.8,4.2
2,30,60,123.3,85.0,35.9,2.4
3,30,60,126.8,86.4,34.7,5.6
4,30,60,122.8,84.1,36.0,2.8
5,30,60,125.9,87.6,34.6,3.7
6,30,60,120.8,82.6,35.7,2.5
7,30,60,119.7,81.4,34.4,3.9
8,30,60,129.6,88.8,36.0,4.8
9,30,60,124.4,84.9,35.7,3.8


We want to compare $k$ systems (given by every combination of reorder points and order sizes) and select a system with the smallest total cost $\mu_{i1}$. We want $P(CS) \geq P^*$ provided that $\mu_{i2} - \mu_{i1} \geq d^*$, where the minimal CS probability $P^* > \frac{1}{k} $ and the “indifference” amount $d^* > 0$ are both specifi ed by the analyst.

The statistical procedure for solving this problem involves “two-stage” sampling from each of the $k$ systems. In the fi rst stage we make a fi xed number of replications of each system, then use the resulting variance estimates to determine how many more replications from each system are necessary in a second stage of sampling in order to reach a decision.

In the first-stage sampling, we make $n_0 \geq 2$ replications of each of the $k$ systems and define the first-stage sample means and variances
$$
\bar{X}_i^{(1)}\left(n_0\right)=\frac{\sum_{j=1}^{n_0} X_{i j}}{n_0}
$$
and
$$
S_i^2\left(n_0\right)=\frac{\sum_{j=1}^{n_0}\left[X_{i j}-\bar{X}_i^{(1)}\left(n_0\right)\right]^2}{n_0-1}
$$
for $i=1,2, \ldots, k$. 

In [38]:
reorder_points = [10,20]
order_sizes = [50,60,80,100]
n_0 = 20

res_stage1 = pd.DataFrame(model.run_experiments(reorder_points, order_sizes, num_rep=n_0))
res = res_stage1.groupby(['reorder_point', 'order_size'])['total_cost'].agg(["mean", "var"]).reset_index()
res

Iteration 100 of 160


,reorder_point,order_size,mean,var
0,10,50,122.950,40.193158
1,10,60,123.325,11.419868
2,10,80,127.250,13.568947
3,10,100,132.825,9.423026
4,20,50,118.690,9.518842
5,20,60,121.325,10.171447
6,20,80,127.160,6.423579
7,20,100,134.395,5.454184


Then we compute the total sample size $N_i$ needed for system $i$ as
$$
N_i=\max \left\{n_0+1,\left\lceil\frac{h_1^2 S_i^2\left(n_0\right)}{\left(d^*\right)^2}\right\rceil\right\}
$$
where $\lceil x\rceil$ is the smallest integer that is greater than or equal to the real number $x$, and $h_1$ (which depends on $k, P^*$, and $n_0$ ) is a constant that can be obtained from a table.

In [39]:
d_star = 1.0
P_star = 0.95
h_1 = 3.551

In [40]:
res['N_i'] = res.apply(lambda row: max(np.ceil((h_1**2)*row["var"]/d_star**2), n_0 + 1), axis=1).astype(int)

display(res)

,reorder_point,order_size,mean,var,N_i
0,10,50,122.950,40.193158,507
1,10,60,123.325,11.419868,144
2,10,80,127.250,13.568947,172
3,10,100,132.825,9.423026,119
4,20,50,118.690,9.518842,121
5,20,60,121.325,10.171447,129
6,20,80,127.160,6.423579,81
7,20,100,134.395,5.454184,69


Next, we make $N_i-n_0$ more replications of system $i(i=1,2, \ldots, k)$ and obtain the second-stage sample means
$$
\bar{X}_i^{(2)}\left(N_i-n_0\right)=\frac{\sum_{j=n_0+1}^{N_i} X_{i j}}{N_i-n_0}
$$

In [41]:
res_stage2 = []

for i in range(res.shape[0]):
    df = pd.DataFrame(model.run_experiments([res['reorder_point'][i]], [res['order_size'][i]], res['N_i'][i]-n_0))
    mean = df['total_cost'].mean()
    res_stage2.append(mean)
    
# print(res_stage2)

Iteration 100 of 487
Iteration 200 of 487
Iteration 300 of 487
Iteration 400 of 487
Iteration 100 of 124
Iteration 100 of 152
Iteration 100 of 101
Iteration 100 of 109


In [45]:
res['mean(N_i - n_0)'] = res_stage2
display(res)

,reorder_point,order_size,mean,var,N_i,mean(N_i - n_0),W_i1,W_i2,X_tilda
0,10,50,122.950,40.193158,507,124.312320,0.043119,0.956881,124.253578
1,10,60,123.325,11.419868,144,124.007258,0.139003,0.860997,123.912422
2,10,80,127.250,13.568947,172,126.473684,0.139541,0.860459,126.582012
3,10,100,132.825,9.423026,119,132.095960,0.182597,0.817403,132.229080
4,20,50,118.690,9.518842,121,119.443564,0.198701,0.801299,119.293830
5,20,60,121.325,10.171447,129,121.044037,0.182570,0.817430,121.095332
6,20,80,127.160,6.423579,81,126.645902,0.248596,0.751404,126.773704
7,20,100,134.395,5.454184,69,133.902041,0.315800,0.684200,134.057717


Then define the weights
$$
W_{i 1}=\frac{n_0}{N_i}\left\{1+\sqrt{1-\frac{N_i}{n_0}\left[1-\frac{\left(N_i-n_0\right)\left(d^*\right)^2}{h_1^2 S_i^2\left(n_0\right)}\right]}\right\}
$$
and $W_{i 2}=1-W_{i 1}$, for $i=1,2, \ldots, k$. Finally, define the weighted sample means
$$
\tilde{X}_i\left(N_i\right)=W_{i 1} \bar{X}_i^{(1)}\left(n_0\right)+W_{i 2} \bar{X}_i^{(2)}\left(N_i-n_0\right)
$$
and select the system with the smallest $\tilde{X}_i\left(N_i\right)$.

In [43]:
res['square_brac'] = 1-(res['N_i']-n_0)*d_star**2/(h_1**2*res['var'])
res['square_root_arg'] = 1 - res['N_i']/n_0*res['square_brac'] 
res['square_root'] = np.sqrt(res['square_root_arg'])
res['W_i1'] = (n_0/res['N_i'])*(1+res['square_root'])
res['W_i2'] = 1-res['W_i1']
res['X_tilda'] = res['W_i1']*res['mean'] + res['W_i2']*res['mean(N_i - n_0)']
res = res.drop(['square_brac', 'square_root_arg','square_root'], axis=1)
display(res)

,reorder_point,order_size,mean,var,N_i,mean(N_i - n_0),W_i1,W_i2,X_tilda
0,10,50,122.950,40.193158,507,124.312320,0.043119,0.956881,124.253578
1,10,60,123.325,11.419868,144,124.007258,0.139003,0.860997,123.912422
2,10,80,127.250,13.568947,172,126.473684,0.139541,0.860459,126.582012
3,10,100,132.825,9.423026,119,132.095960,0.182597,0.817403,132.229080
4,20,50,118.690,9.518842,121,119.443564,0.198701,0.801299,119.293830
5,20,60,121.325,10.171447,129,121.044037,0.182570,0.817430,121.095332
6,20,80,127.160,6.423579,81,126.645902,0.248596,0.751404,126.773704
7,20,100,134.395,5.454184,69,133.902041,0.315800,0.684200,134.057717


In [44]:
res.sort_values(by=['X_tilda'])

,reorder_point,order_size,mean,var,N_i,mean(N_i - n_0),W_i1,W_i2,X_tilda
4,20,50,118.690,9.518842,121,119.443564,0.198701,0.801299,119.293830
5,20,60,121.325,10.171447,129,121.044037,0.182570,0.817430,121.095332
1,10,60,123.325,11.419868,144,124.007258,0.139003,0.860997,123.912422
0,10,50,122.950,40.193158,507,124.312320,0.043119,0.956881,124.253578
2,10,80,127.250,13.568947,172,126.473684,0.139541,0.860459,126.582012
6,20,80,127.160,6.423579,81,126.645902,0.248596,0.751404,126.773704
3,10,100,132.825,9.423026,119,132.095960,0.182597,0.817403,132.229080
7,20,100,134.395,5.454184,69,133.902041,0.315800,0.684200,134.057717
